In [0]:
account_key = dbutils.secrets.get(scope="formala1-scope", key="account-key")
spark.conf.set("fs.azure.account.key.formala1datalake.dfs.core.windows.net", account_key)

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date = dbutils.widgets.get("p_file_date")
print(v_file_date)

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType,DataType

In [0]:
name_schema = StructType(fields=[StructField("forename",StringType(),True),
                          StructField("surname",StringType(),True)])

drivers_schema = StructType(fields=[StructField("driverId",IntegerType(),False),
                                    StructField("driverRef",StringType(),True),
                                    StructField("number",IntegerType(),True),
                                    StructField("code",StringType(),True),
                                    StructField("name",name_schema,True),
                                    StructField("dob",StringType(),True),
                                    StructField("nationality",StringType(),True),
                                    StructField("url",StringType(),True)])

In [0]:
drivers_df = spark.read.json(f"abfss://raw@formala1datalake.dfs.core.windows.net/{v_file_date}/drivers.json",schema=drivers_schema)

In [0]:
display(drivers_df)

In [0]:
drivers_df.printSchema()

In [0]:
from pyspark.sql.functions import col, current_timestamp, concat, lit
drivers_df = drivers_df.withColumnRenamed("driverId","driver_id").withColumnRenamed("driverRef","driver_ref").withColumn("ingestion_date",current_timestamp()).withColumn("name",concat(col("name.forename"),lit(" "),col("name.surname"))).withColumn("file_date",lit(v_file_date))

In [0]:
display(drivers_df)

In [0]:
drivers_df=drivers_df.drop("url")

In [0]:
drivers_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.drivers")

In [0]:
%sql
SELECT * FROM f1_processed.drivers
order by file_date desc